In [1]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier nltk

In [2]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client

import nltk
from nltk.stem import PorterStemmer
!nltk.download('punkt')

import pyterrier as pt
import pandas as pd

/bin/bash: -c: line 1: syntax error near unexpected token `'punkt''
/bin/bash: -c: line 1: `nltk.download('punkt')'


In [3]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

stemmer = PorterStemmer()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

In [5]:
# Get topics and stem them
topics = pt_dataset.get_topics('text')

stemmed_topics = []
for idx, row in topics.iterrows():
    stemmed_query = ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(row['query'])])
    stemmed_topics.append({'qid': row['qid'], 'query': stemmed_query})

In [6]:
# Define the retrieval pipeline with BM25
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

# Convert stemmed_topics to DataFrame
df_stemmed_topics = pd.DataFrame(stemmed_topics)

In [7]:
run = bm25(df_stemmed_topics)

In [8]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".


In [9]:
# #Localtest
# pt.Experiment(
#     [bm25], 
#     pt_dataset.get_topics(), 
#     pt_dataset.get_qrels(), 
#     eval_metrics=['P_1000', 'map', 'recip_rank'],
#     names=['BM25'],
#     baseline=0
#     )

In [10]:
# pt.Experiment(
#     [bm25], 
#     df_stemmed_topics, 
#     pt_dataset.get_qrels(), 
#     eval_metrics=['P_1000', 'map', 'recip_rank'],
#     names=['BM25 + Porter2 Stemmer'],
#     baseline=0
# )